# Tic-tac-toe minmax algorithm with alpha beta pruning

In [21]:
import numpy as np
import copy
import time


The original code has been extended with the implementation of the minmax algorithm and with alpha beta pruning.

The **minmax** method has been changed to now have alpha, beta

In [24]:
class State:
    """ Capturing the state of the game
    gameplan - 3x3 array (0: Empty, 1: X, 2: O)
    player - The player whose turn it is in the game.
    current_player - The player currently being evaluated in the search tree.
    depth - Current depth in the search tree.
    max_depth - Maximum search limit.
    """

    generated = 0

    def __init__(self, gameplan, player, current_player=None, depth=0, max_depth=100):
        self.gameplan = np.copy(gameplan) 
        self.player = player
        if current_player is None:
            self.current_player = player
        else:
            self.current_player = current_player
        self.depth = depth
        self.max_depth = max_depth
        
        State.generated += 1

    def terminal_test(self):
        """ The method tests the current state and returns the winner if the game is over.
            0 - Draw / Game continues
            1 - Player 1 wins
            2 - Player 2 wins
        """
        # Winning conditions (Rows, Columns)
        for i in range(3):
            if np.array_equal(self.gameplan[i], [1, 1, 1]): return 1
            if np.array_equal(self.gameplan[i], [2, 2, 2]): return 2
            if np.array_equal(self.gameplan[:, i], [1, 1, 1]): return 1
            if np.array_equal(self.gameplan[:, i], [2, 2, 2]): return 2
        
        # Winning conditions (Diagonals)
        if np.array_equal(self.gameplan.diagonal(), [1, 1, 1]): return 1
        if np.array_equal(self.gameplan.diagonal(), [2, 2, 2]): return 2
        if np.array_equal(np.fliplr(self.gameplan).diagonal(), [1, 1, 1]): return 1
        if np.array_equal(np.fliplr(self.gameplan).diagonal(), [2, 2, 2]): return 2

        # Draw check
        if 0 not in self.gameplan:
            return 0 # Draw
        return 0 # Game continues

    def utility(self, result):
        """ The method evaluates the current state from the perspective of self.player.
            1: self.player wins, -1: Opponent wins, 0: Draw
        """
        if result == 0: return 0
        elif result == self.player: return 1
        else: return -1

    def possible_actions(self):
        """ The method returns a list of possible actions (empty coordinates). """
        actions = []
        for i in range(3):
            for j in range(3):
                if self.gameplan[i][j] == 0:
                    actions.append((i, j))
        return actions

    def expand(self, select_action):
        """ Creates a new game state by applying the selected action. """
        # Coordinate checks
        if select_action[0] not in range(3) or select_action[1] not in range(3): return None
        if self.gameplan[select_action[0], select_action[1]] != 0: return None
        
        new_array = np.copy(self.gameplan)
        new_array[select_action[0], select_action[1]] = self.current_player
        
        # New state is returned with the opponent as the next current_player
        return State(new_array, 
                     self.player, 
                     self.next_current_player(), 
                     self.depth + 1, 
                     max_depth=self.max_depth)

    def minmax(self, strategy="max", alfa=float('-inf'), beta= float('inf')):
        """
        Minimax algorithm implementation with Alpha-Beta pruning and depth limit.
        """
        
        actions = self.possible_actions()
        result = self.terminal_test()
        
        # 1. BASE CASE: TERMINAL TEST OR DEPTH LIMIT
        if result != 0 or self.depth >= self.max_depth: 
            # If game is over OR Max Depth is reached
            # Score is calculated. Action is insignificant, return an available action.
            score = self.utility(result) if result != 0 else 0 
            return score, actions[0] if actions else (0, 0)


        # 2. INITIALIZATION
        if strategy == "max":
            selected_utilization_value = float('-inf')
            next_strategy = "min"
        else:
            selected_utilization_value = float('inf')
            next_strategy = "max"

        if not actions: 
             return 0, (0, 0) # Should be caught by terminal_test

        selected_action = actions[0]

        # 3. RECURSIVE SEARCH WITH PRUNING
        for action in actions:
            expanded_state = self.expand(action)
            
            # Recursive call (passing updated alpha and beta)
            utilization, _ = expanded_state.minmax(next_strategy, alfa, beta)
            
            # MAXIMIZER (Player X)
            if strategy == "max":
                if utilization > selected_utilization_value:
                    selected_utilization_value = utilization
                    selected_action = action
                
                # ALPHA UPDATE
                alfa = max(alfa, selected_utilization_value)
                
                # BETA PRUNING (Cutoff)
                if selected_utilization_value >= beta:
                    return selected_utilization_value, selected_action
            
            # MINIMIZER (Player O)
            else: # strategy == "min"
                if utilization < selected_utilization_value:
                    selected_utilization_value = utilization
                    selected_action = action
                    
                # BETA UPDATE
                beta = min(beta, selected_utilization_value)

                # ALPHA PRUNING (Cutoff)
                if selected_utilization_value <= alfa:
                    return selected_utilization_value, selected_action
            
        return selected_utilization_value, selected_action

    def next_current_player(self):
        """ Returns the opponent for the purpose of traversing the search space. """
        return 3 - self.current_player

    def next_player(self):
        """ Returns the opponent for switching the game turn. """
        return 3 - self.player

# Game of tic-tac-toe

Creating the initial state of the game
* Game plan is empty (0)
* Game 1 is on the turn

In [25]:
state = State(gameplan=np.array([[0, 0, 0], [0, 0, 0], [0, 0, 0]]),
              player=1)

The cycle of the game remained unchanged. 

Run the turn and compare the result of the turn, times and numbers of generated states against the original minmax version.

In [ ]:
# TEST FUNCTION
def run_game(max_d):
    """
    Runs a game of Tic-Tac-Toe using the Minimax algorithm with a specified 
    maximum search depth (max_d), observing game result, time, and states generated.
    """
    # Reset generation count and timer
    State.generated = 0
    start_time = time.time()
    
    # Initialize the game state. P1 starts.
    state = State(gameplan=np.array([[0, 0, 0], [0, 0, 0], [0, 0, 0]]),
                  player=1, 
                  current_player=1, # Initial player makes the first move
                  max_depth=max_d)
    
    print(f"\n--- Running Game (Max Depth: {max_d}) ---")
    
    while True:
        # 1. PRE-MOVE CHECK
        game_result = state.terminal_test()
        actions_available = len(state.possible_actions())
        
        if game_result != 0 or actions_available == 0:
            winner_text = "Drawn" if game_result == 0 else f'Winner is {game_result}'
            print(f"Result: {winner_text}")
            break

        current_player = state.player
        
        # Set the state's internal evaluation perspective (self.player) to the current player 
        # so that utility results are calculated from *their* viewpoint (+1 is a win for them).
        original_eval_player = state.player 
        state.player = current_player 
        
        # Set the current_player for the search root
        state.current_player = current_player 

        # Minimax always maximizes the outcome for the player whose turn it is.
        util, player_action = state.minmax("max")
        
        # Restore the permanent evaluation player (P1 in the original game setup)
        state.player = original_eval_player 

        # 3. APPLY MOVE (Uses state.current_player to determine the piece to place)
        state = state.expand(player_action) 
        
        print(f"Player {current_player} move: {player_action} -> Utility (for P{current_player}): {util}")
        print(state.gameplan)
        
        # 4. SWITCH TURN
        # Switch the actual game turn for the next iteration.
        state.player = state.next_player() 
        # Set the next current_player for the next turn's root search.
        state.current_player = state.player 

    # 5. REPORT RESULTS
    end_time = time.time()
    print(f"Total States Generated: {State.generated}")
    print(f"Total Time: {end_time - start_time:.4f}s")
    print("---------------------------------------")


# Run the game with different search depth constraints to compare performance.
run_game(max_d=9) # Max Depth 9: Full Search (Optimal) 
run_game(max_d=4) # Constrained Depth 4
run_game(max_d=2) # Very Limited Depth 2


--- Running Game (Max Depth: 9) ---
Player 1 move: (0, 0) -> Utility (for P1): 0
[[1 0 0]
 [0 0 0]
 [0 0 0]]
Player 2 move: (1, 1) -> Utility (for P2): 0
[[1 0 0]
 [0 2 0]
 [0 0 0]]
Player 1 move: (0, 1) -> Utility (for P1): 0
[[1 1 0]
 [0 2 0]
 [0 0 0]]
Player 2 move: (0, 2) -> Utility (for P2): 0
[[1 1 2]
 [0 2 0]
 [0 0 0]]
Player 1 move: (2, 0) -> Utility (for P1): 0
[[1 1 2]
 [0 2 0]
 [1 0 0]]
Player 2 move: (1, 0) -> Utility (for P2): 0
[[1 1 2]
 [2 2 0]
 [1 0 0]]
Player 1 move: (1, 2) -> Utility (for P1): 0
[[1 1 2]
 [2 2 1]
 [1 0 0]]
Player 2 move: (2, 1) -> Utility (for P2): 0
[[1 1 2]
 [2 2 1]
 [1 2 0]]
Player 1 move: (2, 2) -> Utility (for P1): 0
[[1 1 2]
 [2 2 1]
 [1 2 1]]
Result: Drawn
Total States Generated: 21653
Total Time: 0.9423s
---------------------------------------

--- Running Game (Max Depth: 4) ---
Player 1 move: (0, 0) -> Utility (for P1): 0
[[1 0 0]
 [0 0 0]
 [0 0 0]]
Player 2 move: (0, 1) -> Utility (for P2): 0
[[1 2 0]
 [0 0 0]
 [0 0 0]]
Player 1 move: (0, 

# Task
Add a constraint to the algorithm to limit the maximum search depth.

Again, you need to change the code in place of # **!!! todo**

In [27]:
state = State(gameplan=np.array([[0, 0, 0], [0, 0, 0], [0, 0, 0]]),
              player=1, max_depth=2)

In [29]:
while True:
    # Check that the game is not over
    game_result = state.terminal_test()
    if game_result != 0:
        print(f"Winner is {game_result} ")
        break

    # Checking for drawn
    if len(state.possible_actions()) == 0:
        print("Drawn")
        break

    # player's move
    print(f"=====================\nPlayer {state.player}")
    _, player_action = state.minmax("max")
    print(f"Select action: {player_action}")
    state = state.expand(player_action)
    print(state.gameplan)
    print(f"Generated states {State.generated}.")
    State.generated = 0

    # switching the game to the other player
    state.player = state.next_player()

Winner is 1 
